# SPECFEM Users Workshop -- Day 3 (Oct. 7, 2022)

## Part 3: Seismic Imaging with SeisFlows + Pyatoa

In this notebook we will bring all of the previous material together run a full seismic inversion to update a starting model. 

-----------

### Relevant Information

>__NOTE:__ These instructions should be run from inside the Docker container, using Jupyter Lab. The Docker container should have the adjTomo toolkit installed (SeisFlows, Pyatoa, PySEP), as well as SPECFEM2D and SPECEFM3D compiled with MPI. 

**Relevant Links:** 
- Day 2 Slides !!! ADD THIS !!!
- [Today's Notebook](https://github.com/adjtomo/adjdocs/blob/main/workshops/2022-10-05_specfem_users/day_2a_kernels.ipynb)
- Completed Notebook !!! ADD THIS !!!

**Jupyter Quick Tips:**

- **Run cells** one-by-one by hitting the $\blacktriangleright$ button at the top, or by hitting `Shift + Enter`
- **Run all cells** by hitting the $\blacktriangleright\blacktriangleright$ button at the top, or by running `Run -> Run All Cells`
- **Currently running cells** that are still processing will have a `[*]` symbol next to them
- **Finished cells** will have a `[1]` symbol next to them. The number inside the brackets represents what order this cell has been run in.
- Commands that start with `!` are Bash commands (i.e., commands you would run from the terminal)
- Commands that start with `%` are Jupyter Magic commands.


## 1) Background

!!! TO DO !!!
Potential topics:
- Nonlinear optimization algorithms
- Line searches
- Model perturbations

## 2) Setting Up 

As with Day 1, we will want to set up a clean working directory to run SPECFEM2D inside. This will help us preserve our cloned repository and reduce file clutter.

>__NOTE:__ We will be doing all our work in the directory /home/scoped/work_day_2. All the following cells assume that we are in this directory, so you must evaluate the '%cd' command to ensure that cells work as expected.

In [ ]:
# make sure we're in an empty working directory
! mkdir -p /home/scoped/work/day_3/manual_update
%cd /home/scoped/work/day_3/manual_update

# Run the example and stop after adjoint simulation
! seisflows examples setup 2 -r /home/scoped/specfem2d --event_id 1 --nsta 1 --niter 1 --with_mpi
! seisflows par stop_after evaluate_gradient_from_kernels
! seisflows submit

TO DO:
- Run a manual model update by guessing the perturbation as the negative gradient? Try to follow golden rule or smoething
- Attempt one or two updates manually, checking waveform misfit each time
- Then run Example 1 to show HH models
- Run Example 2 for inversion
- Allow users to play around with starting model, preprocessing etc.

In [ ]:
! seisflows plot2d GRADIENT_01 vs_kernel --savefig g_01_vs.png

In [ ]:
from IPython.display import Image
Image("g_01_vs.png")

## 3) Manually Updating a Velocity Model

We can use this gradient that was generated using SeisFlows (and manually in yesterday's notebook), to update our initial model. 
Since we only have one source receiver pair, we can immediately check if the misfit of the waveforms has decreased through our update procedure.
To do this we treat our models as linear arrays, and add them together

In [ ]:
from seisflows.tools.specfem import Model

m_init = Model("output/MODEL_INIT")
print(f"INITIAL MODEL\n{m_init.model}")
m_init.plot2d("vs")

In [ ]:
# Because the Gradient output directory does not contain coordinate information,
# we need to grab it from the model, which natively exports to disk with 
# coordinate information
gradient = Model("output/GRADIENT_01")
gradient.coordinates = {}
gradient.coordinates["x"] = m_init.coordinates["x"]
gradient.coordinates["z"] = m_init.coordinates["z"]

print(f"GRADIENT\n{gradient.model}")
gradient.plot2d("vs_kernel")

In [ ]:
# We can use SeisFlows to update the model directly
m_update = m_init.copy()
m_update.update(vector=m_init.vector - gradient.vector)

print(m_update.model)

m_update.plot2d("vs")

We can see that because the gradient is **not** well scaled, the model updates only make slight changes to the Vs model (looking at the title). In order to make appreciable changes to the starting model, we need the **scale** the gradient. There are many algorithms which provide scaling estimates for the gradient. One thing we can try, is scaling by GTG$^-1$, or the inverse of the dot product of the gradient itself.

In [ ]:
import numpy as np

gtg = np.dot(gradient.vector, gradient.vector)
gtg_inverse = gtg ** -1 

print(gtg_inverse)

In [ ]:
# We can use SeisFlows to update the model directly
m_update = m_init.copy()
m_update.update(vector=m_init.vector - (gtg_inverse * gradient.vector))

print(m_update.model)

m_update.plot2d("vs")

We can see that our gradient is now more well scaled, and has updated our velocity model by at most 100m/s. We can now try to use this updated velocity model to generate **new** synthetics, and calculate waveform misfit. If misfit has reduced from the original model, then we're getting somewhere!